**Exercise 8** 
This is the code to compute the likelihood function of exercise 8 of problem set 3. 


In [125]:
using StatFiles, DataFrames

data_path = "/Users/jorditorresvallverdu/Downloads/xsection_05.dta"

export_dir = "/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/policy_effects/figures/"


main = DataFrame(load(data_path))

Row,occup,d_months,hwage
,Float32?,Float32?,Float32?
1,3.0,10.0,47.619
2,2.0,1.0,18.4854
3,2.0,7.0,22.5694
4,3.0,244.0,21.3294
5,3.0,88.0,7.93651
6,3.0,124.0,16.7143
7,3.0,136.0,21.645
8,1.0,0.0,missing
9,3.0,2.0,25.5952


In [ ]:
##Code tbe parameters of the likelihood function
using Distributions

function log_likelihood(θ, w_i_star,w_f_star,t_u, w_i, w_f,u, e_i, e_f)

μ_i, μ_f, σ_i, σ_f, λ_i, λ_f, η_i, η_f = unpack_params(θ)


Fi_star= cdf.(LogNormal(μ_i, σ_i), w_i_star) #need to define w_i_star properly following Heckman
Ff_star= cdf.(LogNormal(μ_f, σ_f), w_f_star) #need to define w_f_star outside too

F_i_tilde=  1-Fi_star
F_f_tilde=  1-Ff_star

a_i=  λ_i* F_i_tilde
a_f=  λ_f* F_f_tilde

h_u=  a_i + a_f

p_u= (1+(a_i/η_i)+(a_f/η_f))^-1
p_ei= (a_i/η_i)/(1+(a_i/η_i)+(a_f/η_f))
p_ef=  (a_f/η_f)/(1+(a_i/η_i)+(a_f/η_f))

f_i= pdf.(LogNormal(μ_i, σ_i), w_i) ##need to define w_i as input
f_f= pdf.(LogNormal(μ_f, σ_f), w_f)


f_i_full[e_i] .= f_i
f_f_full[e_f] .= f_f


ll_u= @. u*(log(h_u) - h_u* t_u+ log(p_u))
ll_ei= @. e_i *(log(η_i)+(-η_i*t_u) + log(f_i_full)- log(F_i_tilde)+ log(p_ei)) 
ll_ef= @. e_f*(log(η_f)+(-η_f*t_u) + (log(f_f_full))- log(F_f_tilde)+ log(p_ef))


    return -(sum(ll_u) + sum(ll_ef) + sum(ll_ei))
end


log_likelihood (generic function with 4 methods)

In [ ]:
##define parameters/inputs of the function 
u= (main.occup .== 1) #vector
e_f= (main.occup .==2) #vector
e_i= (main.occup .==3) #vector

t_u=main.d_months #vector


w_i=main.hwage[e_i] #vector
w_f=main.hwage[e_f] #vector 

w_i_star= minimum(w_i) #scalar
w_f_star= minimum(w_f) #scalar



5.796326637268066

In [145]:
function unpack_params(θ::AbstractVector{T}) where T
    μ_i = θ[1]
    μ_f = θ[2]

    σ_i = exp(θ[3])
    σ_f = exp(θ[4])

    λ_i = exp(θ[5])
    λ_f = exp(θ[6])

    η_i = exp(θ[7])
    η_f = exp(θ[8])

    return μ_i, μ_f, σ_i, σ_f, λ_i, λ_f, η_i, η_f
end



unpack_params (generic function with 2 methods)

In [129]:
θ0 = [
    mean(log.(w_i)),             # μ_i
    mean(log.(w_f)),             # μ_f
    log(std(log.(w_i))),         # log σ_i
    log(std(log.(w_f))),         # log σ_f
    log(0.2),                    # log λ_i
    log(0.2),                    # log λ_f
    log(0.05),                   # log η_i
    log(0.05)                    # log η_f
]

obj(θ) = log_likelihood(θ,
    w_i_star, w_f_star,
    t_u, w_i, w_f,
    u, e_i, e_f
)

obj (generic function with 1 method)

In [149]:
##Optimize
using Optim

res = optimize(obj, θ0, BFGS())
θ_hat = Optim.minimizer(res)
LL_min = Optim.minimum(res)

println("Estimated parameters θ̂ = ", θ_hat)
println("Minimum negative log-likelihood = ", LL_min)



Estimated parameters θ̂ = [2.880490362258286, 3.136134297037597, -0.5454886759741853, -0.6474618101753752, -2.01616349957599, -1.855731749224869, -4.598597454283934, -4.57016838015646]
Minimum negative log-likelihood = 150136.1597622516


In [ ]:
using FiniteDiff

H_hat = FiniteDiff.finite_difference_hessian(obj, θ_hat)

V = inv(H_hat)
se = sqrt.(diag(V))

println("Standard errors = ", se)

H_hat


Standard errors = [0.007088607635006738, 0.006088249157826606, 0.008851087853149609, 0.00867642182749136, 0.034093746484922666, 0.03350015601637799, 0.012088871292023908, 0.011322234148005242]


8×8 Symmetric{Float64, Matrix{Float64}}:
 19914.9          -0.465709  …    -19.5664    -21.8912     20.5259
    -0.465709  27139.9             92.5506     83.5061    -89.8247
   416.938         0.833437        35.0203     39.182     -36.7379
     0.642116   1455.99          -127.62     -115.147     123.86
    22.3242      -79.6029       -3344.36    -3741.73     3508.34
   -19.5664       92.5506    …   3888.33     3508.34    -3773.81
   -21.8912       83.5061        3508.34    10567.8     -3508.34
    20.5259      -89.8247       -3773.81    -3508.34    11538.2

In [152]:
using ForwardDiff, LinearAlgebra

f = θ-> obj(θ) 
H = θ -> ForwardDiff.hessian(f, θ)
H_hat= H(θ_hat)

V= inv(H_hat)
se= sqrt.(diag(V))

true

In [100]:
using DataFrames

varnames=["μ_i", "μ_f", "σ_i", "σ_f", "λ_i", "λ_f", "η_i", "η_f"]

results_df = DataFrame(
        term = ["β$i ($name)" for (i, name) in enumerate(varnames)], ##quite nice tool to actually loop in a more efficient way over order of variables and variable names. 
        estimate = θ_hat,
        std_error = se
    )



Row,term,estimate,std_error
,String,Float64,Float64
1,β1 (μ_i),2.88049,0.00708861
2,β2 (μ_f),3.13613,0.00608825
3,β3 (σ_i),-0.545489,0.00885109
4,β4 (σ_f),-0.647462,0.00867642
5,β5 (λ_i),-2.01616,0.0340937
6,β6 (λ_f),-1.85573,0.0335002
7,β7 (η_i),-4.5986,0.0120889
8,β8 (η_f),-4.57017,0.0113222


In [153]:
exp(-4.5986)

0.010065918170073745